### Task 3: Adding a Pedestrian in the "Corner Scenario"

This notebook can be used to add a pedestrian in the corner of the "corner scenario" defined in the RiMEA guideline<sup>1</sup>. The scenario file, which can be used by the simulation software Vadere<sup>2</sup>, should already exist and contain all information stated in the guideline.
Note: In general, this notebook can be used to add a pedestrian to any scenario.

In [ ]:
# Imports
from JSON_utils import *
from scenario_modification import *
import os
import numpy as np

Some values need to be entered manually. They are the following:
1. The path to the scenario file.
2. The path to the file the new scenario is stored. If none is entered, the scenario will be stored in the same folder as the original scenario file. Additionally, "_modified" will be added to the file name. Note that the output file will be overwritten if it already exists.
3. The coordinates of the pedestrian's starting position
4. A pair of coordinates inside the target the pedestrian should reach. It is required to determine the target's ID. The function determining the IDs requires a list of coordinates to be able to search for a set of targets.

The default values are chosen that they work with the scenario used in task 1 and the template's folder structure.

In [ ]:
scenario_file = "./scenarios/MI.scenario"
output_file = None
pedestrian_coordinates_array = np.loadtxt("pedestrian_pos.txt")
pedestrian_coordinates = [tuple(pair) for pair in pedestrian_coordinates_array]
target_coordinates = [(183.5, 103.5)]

In the next cell, some information is gathered which is necessary to create a pedestrian. This inludes:
1. The ID of the target they try to reach.
2. The ID that will be assigned to the pedestrian.
3. The attributes of pedestrians. They should apply to the new pedestrian as well.
4. The seed that will be used to randomly sample the pedestrian's speed.

In [ ]:
# Parse the scenario file
scenario = parse_json(scenario_file)

# Retrieve the ID of the target
targets = get_targets(scenario)
target_id = find_target_at_position(targets, target_coordinates)

if not target_id:
    print("No target areas including the coordinates were found. Please check the coordinates and scenario.")

Now, the pedestrian can be created. Therefore, it is verified that the pedestrian will be placed inside the simulation area and not inside an obstacle.

In [ ]:
pedestrians = []
count = 0
for pedestrian_coordinate in pedestrian_coordinates:
    if verify_coordinates(scenario, pedestrian_coordinate):
        # Determine the pedestrian's ID
        pedestrian_id = get_next_free_id(scenario)
        
        # Retrieve pedestrians' attributes.
        attributes = get_attributes(scenario)
        
        # Retrieve the seed.
        seed = get_seed(scenario)
        
        pedestrian = create_pedestrian(pedestrian_coordinate, pedestrian_id, target_id, attributes, seed)
        pedestrians.append(pedestrian)
    else:
        count += 1
        print("The coordinates at which the pedestrian should be placed are either out of bounds or blocked by an obstacle.")
        print("Please check the coordinates and the scenario.")

print(f"A total of {count} pedestrians could not be placed.")

Finally, the pedestrian can be added to the simulation. Before converting the scenario back to a JSON file, the name needs to be changed. Otherwise, this will cause problems in the Vadere GUI.

In [ ]:
for pedestrian in pedestrians:
    scenario = add_pedestrian(pedestrian, scenario)

# Requires Python 3.9 or newer
if not output_file:
    output_file = scenario_file.removesuffix(".scenario") + "_modified.scenario"
    new_name = None
else:
    new_name = output_file.split("/")[-1].removesuffix(".scenario")
    
scenario = adjust_name(new_name, scenario)

# Convert to JSON file
write_json(output_file, scenario)

We can now use the updated scenario file to run the simulation. Instead of executing the next cell, of course, one could also use the GUI to execute the adapted scenario. The default location of Vadere's jar file and the output directory is coherent with the templates structure and must be adjusted if a different folder structure is used.

In [ ]:
vadere = "../vadere/VadereSimulator/target/vadere-console.jar"
output_dir = "../vadere/Scenarios/output"
_ = os.system("java -jar " + vadere + " scenario-run --scenario-file " + output_file + " --output-dir " + output_dir)

### References
[1] [https://rimea.de/wp-content/uploads/2016/06/rimea_richtlinie_3-0-0_-_d-e.pdf](https://rimea.de/wp-content/uploads/2016/06/rimea_richtlinie_3-0-0_-_d-e.pdf)<br>
[2] [https://www.vadere.org/](https://www.vadere.org/)